This is for a victim with an action space of [0,1]

In [1]:
AGENT_NAME = r'Models/Victim/SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = r'default SAC 500 norm space results' + '/'
EPS = 0.03
ATK_NAME = f'bifurcated_uACG_DLloss_03_mask_time_solar_and_consumption'
CONSUMPTION_IDX = 26
SOLAR_IDX = 24

(M1,B1,M2,B2)

uACG_binary_SAC_03_eps_linear_half_m: (0.5,1.5, -0.5,1) had a higher MAE than (1,0,-1,0) but lower KPIs. Like the discrete case where ASR is not proportional to performance, neither is MAE.

In [2]:
from stable_baselines3 import SAC

from citylearn.data import DataSet
from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedSpaceWrapper, StableBaselines3Wrapper

from art.attacks.evasion import AutoConjugateGradient as ACG
#from KBMproject.mybb import BrendelBethgeAttack as BBA

from art.estimators.classification import PyTorchClassifier as classifier
from art.utils import to_categorical

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

import KBMproject.utilities as utils

import json
from tqdm import tqdm

%matplotlib inline

In [3]:
try: #try to load CityLearn schema
    schema = DataSet.get_schema(DATASET_NAME)
except: #load saved schema otherwise
    with open(DATASET_NAME, 'r') as file:
        schema = json.load(file)

Define RL agent

In [4]:
env = CityLearnEnv(schema, 
                   central_agent=True,
                   buildings=list(schema['buildings'].keys())[0], 
                )
env = NormalizedSpaceWrapper(env)
env = StableBaselines3Wrapper(env)

In [5]:
agent = SAC.load(path=f"{AGENT_NAME}",
                env=env,
                print_system_info=True)
print('Model loaded from storage')


== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.23.5
- Gym: 0.21.0

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\kbrod\AppData\Roaming\Python\Python310\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Model loaded from storage


In [6]:
cols = env.observation_names

In [7]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features
observation_masks[SOLAR_IDX] = 0
observation_masks[CONSUMPTION_IDX] = 0

In [8]:
agent.policy.actor

Actor(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (latent_pi): Sequential(
    (0): Linear(in_features=31, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
  )
  (mu): Linear(in_features=256, out_features=1, bias=True)
  (log_std): Linear(in_features=256, out_features=1, bias=True)
)

In [9]:
from copy import deepcopy
policy_net = deepcopy(agent.actor.latent_pi) #copies shared net rather than referencing/changing the agent
policy_net.add_module('4', agent.actor.mu)
policy_net

Sequential(
  (0): Linear(in_features=31, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=1, bias=True)
)

In [10]:
agent_policy = utils.RegressorLinearWrapper(policy_net)

In [11]:
agent_classifier = classifier(
        model=agent_policy,
        loss=utils.CWLoss(), 
        nb_classes=2,
        input_shape=agent.observation_space.shape,
        device_type='gpu',
        clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max()) #min and max values of each feature, brendle bethge attack only supports floats values and not array
        )

In [12]:
init = 50
iter = int(1000/init)
import logging
logging.basicConfig(level=logging.WARN)
kwargs = {"loss_type": None, 
          "eps": EPS, 
          "eps_step": 2*EPS, 
          "batch_size": 1, 
          "nb_random_init": init, 
          "max_iter": iter, 
          "norm": "inf", 
          "verbose": False}
attack = ACG(estimator=agent_classifier, **kwargs,)

In [13]:
%%time
kpis, obs, adv_obs, actions, adv_actions, mae = utils.eval_continuous_attack(agent, 
                                                                            env, 
                                                                            attack,
                                                                            time_steps=None,
                                                                            mask=observation_masks)

100%|██████████| 8759/8759 [3:52:59<00:00,  1.60s/it, MAE=0.122]


CPU times: total: 12min 22s
Wall time: 3h 53min


In [14]:
display(kpis)

cost_function
annual_peak_average                      1.148366
carbon_emissions_total                   1.040241
cost_total                               0.982513
daily_one_minus_load_factor_average      0.963204
daily_peak_average                       1.141257
electricity_consumption_total            1.051774
monthly_one_minus_load_factor_average    0.997726
ramping_average                          1.852770
zero_net_energy                          1.113353
Name: District, dtype: float64

In [15]:
with open(SAVE_DIR+f'{ATK_NAME} parameters.json', 'w') as f:
    json.dump(kwargs, f)

In [16]:
kpi = kpis

In [17]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

default SAC 500 norm space results/KPIs.csv updated


In [18]:
df_obs = pd.DataFrame(obs)
df_obs.columns = cols
df_obs['a'] = actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+' a-obs.csv')

In [19]:
#pd.DataFrame(adv_actions, columns=['a']).to_csv(SAVE_DIR+ATK_NAME+' adv a.csv')

In [20]:
df_obs = pd.DataFrame(adv_obs)
df_obs.columns = cols
df_obs['a'] = adv_actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+' adv a-obs.csv')

In [21]:
asr_savename = SAVE_DIR+'MAEs.csv'
try:
    df_asrs = pd.read_csv(asr_savename)
    df_asrs[ATK_NAME] = mae
    df_asrs.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    asr = pd.Series([mae])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print(f'{asr_savename} created')

default SAC 500 norm space results/MAEs.csv updated
